Welcome

In [1]:
#dependencies needed for s3.py script
#RUN EVERYTIME YOU CONNECT TO NEW RUN TIME

# !pip install boto3
# !pip install python-dotenv
# !pip install transformers
# !pip install torch

In [ ]:
#imports needed
from util import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import os
from dotenv import load_dotenv
import sys
scripts_folder = os.path.join(os.getcwd(), '..', 'Scripts')
sys.path.append(scripts_folder)
import s3
from time import time

#roberta
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax
import torch
import math
from transformers import RobertaTokenizer, RobertaForSequenceClassification


/uolstore/home/users/sc21aa/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# #connect to google drive to access script and .env file for s3
# #for this to work - you need to add a shortcut to the group project folder to 'MyDrive' (can do this by clicking organise)

# #mount google drive to access required files
# from google.colab import drive
# drive.mount('/content/drive')

# #.env file needed for s3 script
env_path = "/uolstore/home/users/sc21aa/group-project/Scripts/env"
load_dotenv(env_path)
# # #s3 script
# %cd /content/drive/MyDrive/Group_project/Code/
# import s3

False

Change the path to data below as required

In [29]:

s3.download_all("processed/news/gnews_artifacts/")
s3.download("processed/news/gnews_artifacts/AAPL.parquet")
df = pd.read_parquet(s3.s3_to_local_path("processed/news/gnews_artifacts/AAPL.parquet"))
s3.download_all("processed/news/twitter_roberta/")


In [47]:
dfs = [
    (f, pd.read_parquet(f)) for f in s3.s3_to_local_path("processed/news/gnews_artifacts/").glob("*")
]
print(f"{sum(len(df) for _, df in dfs)} rows.")

1742770 rows.


In [ ]:
#RoBERTa model

#connect to GPU
if torch.cuda.is_available():
  device = torch.device("cuda")
  print("Using GPU:", torch.cuda.get_device_name(0))
else:
  device = torch.device("cpu")
  print("Using CPU")

#model name
#MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL).to(device)
print("HH")

# Function to get sentiment scores for a chunk
def get_sentiment_scores(chunk):
    encoded_text = tokenizer(chunk, return_tensors='pt')
    output = model(**encoded_text)
    #extracting and normalising sentiment scores
    scores = output[0][0].detach().numpy()
    #normalising the score
    return softmax(scores)

#function to split chunks if needed since roberta can do 512 at max (max 512 tokens can be processed at a time)
def analyse_large_text(text):
    tokens = tokenizer.encode(text, add_special_tokens=False)  # Encode to token IDs without special tokens
    chunk_size = 512
    # Overlap between chunks to avoid missing context between chunks
    stride = 256
    sentiment_scores = []
    token_lengths = []

    for i in range(0, len(tokens), stride):
        chunk = tokens[i:min(i + chunk_size, len(tokens))]
        chunk_text = tokenizer.decode(chunk)  # Decode back to text

        inputs = tokenizer(chunk_text, return_tensors='pt', truncation=True, max_length=512).to(device)
        output = model(**inputs)
        scores = output[0][0].detach().cpu().numpy()
        sentiment_scores.append(scores)
        token_lengths.append(len(chunk))

    # Weighted average of scores by chunk length
    sentiment_scores = np.array(sentiment_scores)
    weighted_scores = np.average(sentiment_scores, axis=0, weights=token_lengths)
    # print('sentiment scores \n', sentiment_scores)
    # print('weighted scores \n', weighted_scores)
    
    compound_score = weighted_scores[2] - weighted_scores[0]
    
    normalised_compound = compound_score / math.sqrt(compound_score**2 + 20)

    # Return final aggregated sentiment
    return {
        'roberta_pos': weighted_scores[2],
        'roberta_neu': weighted_scores[1],
        'roberta_neg': weighted_scores[0],
        'roberta_compound': compound_score,
        'roberta_normalised_compound': normalised_compound,
    }


start = time()
end = time()
# title_data[['roberta_pos', 'roberta_neu', 'roberta_neg']] = title_data['title'].apply(lambda x: pd.Series(analyse_large_text(x)))
# body_data[['roberta_pos', 'roberta_neu', 'roberta_neg']] = body_data['body'].apply(lambda x: pd.Series(analyse_large_text(x)))

Using GPU: NVIDIA GeForce RTX 4070


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


HH


In [52]:
for path, df in tqdm(dfs):
    out_path = Path(str(path).replace("gnews_artifacts", "twitter_roberta"))
    out_path.parent.mkdir(exist_ok=True, parents=True)
    if not out_path.exists():
        print(out_path)
        df[['roberta_pos', 'roberta_neu', 'roberta_neg', 'roberta_compound', 'roberta_normalised_compound']] = df['text'].apply(lambda x: pd.Series(analyse_large_text(x)))
        df.to_parquet(out_path)

  0%|          | 0/500 [00:00<?, ?it/s]

/local/data/s3local/sentimentgroup/processed/news/twitter_roberta/BR.parquet


  0%|          | 2/500 [00:01<07:57,  1.04it/s]

/local/data/s3local/sentimentgroup/processed/news/twitter_roberta/BRO.parquet


  1%|          | 3/500 [00:02<06:12,  1.33it/s]

...

/local/data/s3local/sentimentgroup/processed/news/twitter_roberta/CB.parquet


 11%|█         | 55/500 [11:44<2:06:12, 17.02s/it]

/local/data/s3local/sentimentgroup/processed/news/twitter_roberta/AMTM.parquet
/local/data/s3local/sentimentgroup/processed/news/twitter_roberta/CBOE.parquet


 11%|█▏        | 57/500 [11:50<1:17:18, 10.47s/it]

/local/data/s3local/sentimentgroup/processed/news/twitter_roberta/AMZN.parquet


In [44]:
s3.upload_all("processed/news/twitter_roberta/")


,url,text,domain,dt,roberta_pos,roberta_neu,roberta_neg,roberta_compound,roberta_normalised_compound
0,https://www.nydailynews.com/2024/11/30/rupert-...,'Harry Potter' star Rupert Grint ordered to pa...,New York Daily News,1732995331000,-1.185435,1.567737,-0.633387,-0.552048,-0.122512
1,https://seekingalpha.com/news/4148049-huawei-t...,Apple (,Seeking Alpha,1725893790000,0.288188,0.790645,-1.643020,1.931208,0.396446
2,https://seekingalpha.com/news/4148049-huawei-t...,Apple (AAPL) and Samsung (OTCPK:SSNLF) to be t...,Seeking Alpha,1725893790000,-0.911968,1.158567,-0.696433,-0.215535,-0.048139
3,https://www.mercurynews.com/2024/06/03/photos-...,Apple CEO’s Woodside mansion for sale for $23 ...,The Mercury News,1717436227000,2.369269,1.082665,-3.268001,5.637269,0.783417
4,https://www.digitaltrends.com/mobile/ups-worke...,Apple gear\nThe desirability and high value of...,Digital Trends,1711007410000,2.807741,0.189212,-2.774343,5.582083,0.780427
...,...,...,...,...,...,...,...,...,...
997,https://www.techspot.com/news/91941-apple-m1-m...,Apple as an MPX module for the Mac Pro and coi...,TechSpot,1635280233000,0.617338,1.236407,-1.763731,2.381070,0.469963
998,https://www.techspot.com/news/91941-apple-m1-m...,"Apple chips, AnandTech’s Andrei Frumusanu has ...",TechSpot,1635280233000,2.665001,-0.230261,-2.614141,5.279142,0.763018
999,https://www.cnet.com/tech/computing/lightroom-...,Apple Photos. The masking ability makes it a b...,CNET,1635279216000,2.457969,0.492762,-2.814759,5.272728,0.762630
1000,https://arstechnica.com/gadgets/2021/10/adobe-...,"Apple Silicon Macs, and new web versions of so...",Ars Technica,1635276183000,0.823065,1.879509,-2.929582,3.752647,0.642796


In [7]:
#Roberta ENSEMBLE MODEL

# # Device setup (GPU if available, otherwise CPU)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Load both sentiment models
# MODEL_1 = "cardiffnlp/twitter-roberta-base-sentiment-latest"  # Social media sentiment model
# MODEL_2 = "ProsusAI/finbert"  # Financial sentiment model

# tokenizer_1 = AutoTokenizer.from_pretrained(MODEL_1)
# model_1 = AutoModelForSequenceClassification.from_pretrained(MODEL_1).to(device)

# tokenizer_2 = AutoTokenizer.from_pretrained(MODEL_2)
# model_2 = AutoModelForSequenceClassification.from_pretrained(MODEL_2).to(device)

# # Function to get sentiment scores for a given model
# def get_sentiment_scores(model, tokenizer, text):
#     inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=512).to(device)
#     output = model(**inputs)
#     scores = output.logits.detach().cpu().numpy()[0]
#     return softmax(scores)  # Normalize the scores

# # Function to analyse large text chunks
# def analyse_large_text(text):
#     tokens = tokenizer_1.encode(text, add_special_tokens=False)
#     chunk_size = 512
#     stride = 256
#     sentiment_scores_1 = []
#     sentiment_scores_2 = []
#     token_lengths = []

#     for i in range(0, len(tokens), stride):
#         chunk = tokens[i:min(i + chunk_size, len(tokens))]
#         chunk_text = tokenizer_1.decode(chunk)

#         # Get sentiment scores from both models
#         scores_1 = get_sentiment_scores(model_1, tokenizer_1, chunk_text)
#         scores_2 = get_sentiment_scores(model_2, tokenizer_2, chunk_text)

#         sentiment_scores_1.append(scores_1)
#         sentiment_scores_2.append(scores_2)
#         token_lengths.append(len(chunk))

#     # Weighted average of scores (based on token lengths)
#     sentiment_scores_1 = np.array(sentiment_scores_1)
#     sentiment_scores_2 = np.array(sentiment_scores_2)

#     weighted_scores_1 = np.average(sentiment_scores_1, axis=0, weights=token_lengths)
#     weighted_scores_2 = np.average(sentiment_scores_2, axis=0, weights=token_lengths)

#     # Final ensemble output (Average scores from both models)
#     final_scores = (weighted_scores_1 + weighted_scores_2) / 2

#     # Return final sentiment scores
#     return {
#         'ensemble_pos': final_scores[2],  # Positive
#         'ensemble_neu': final_scores[1],  # Neutral
#         'ensemble_neg': final_scores[0],  # Negative
#     }

# # Apply sentiment analysis to title and body data
# title_data[['ensemble_pos', 'ensemble_neu', 'ensemble_neg']] = title_data['title'].apply(lambda x: pd.Series(analyse_large_text(x)))
# body_data[['ensemble_pos', 'ensemble_neu', 'ensemble_neg']] = body_data['body'].apply(lambda x: pd.Series(analyse_large_text(x)))


In [8]:
# #save as csv files
# from google.colab import files

# #change file names as required
# title_data.to_csv('GS_title_model3.csv', encoding='utf-8')
# body_data.to_csv('GS_body_model3.csv', encoding='utf-8')

# files.download('GS_title_model3.csv')
# files.download('GS_body_model3.csv')

In [9]:
display(df)
# display(title_data)
# display(body_data)

,title,description,body,url,domain,dt,full_data,roberta_pos,roberta_neu,roberta_neg
0,How short-term vacation rentals use a decades-...,An often-overlooked risk of booking vacation r...,CNN —\nIt has been just over two years since Z...,https://edition.cnn.com/2024/12/24/economy/sho...,Cable News Network,1735034449000,How short-term vacation rentals use a decades-...,0.553966,0.400172,0.045863
1,Watching Home Alone With Macaulay Culkin,‘A Nostalgic Night with Macaulay Culkin’ invit...,Photo-Illustration: Vulture; Photos: 20th Cent...,https://www.vulture.com/article/watching-home-...,Vulture,1734609652000,Watching Home Alone With Macaulay Culkin Photo...,0.800338,0.165531,0.034131
2,How Airbnb is using anti-party technology for ...,Short-term rental giant Airbnb is working to m...,"COLUMBUS, Ohio (WCMH) — Short-term rental gian...",https://fox8.com/news/how-airbnb-is-using-anti...,WJW FOX 8 News Cleveland,1734484684000,How Airbnb is using anti-party technology for ...,0.825245,0.036851,0.137904
3,RollAway’s electric ‘Suite on Wheels’ now avai...,"RollAway, a Silicon Valley startup, is offerin...","While glamping, or glamourous camping, with el...",https://www.digitaltrends.com/cars/rollaway-st...,Digital Trends,1734482059000,RollAway’s electric ‘Suite on Wheels’ now avai...,0.864938,0.010046,0.125016
4,"Airbnb guests destroy home's furniture, redeco...",An Airbnb property manager is seeing red after...,This guest lives rent-free in their heads.\nAn...,https://nypost.com/2024/12/17/lifestyle/i-was-...,New York Post,1734461637000,"Airbnb guests destroy home's furniture, redeco...",0.614556,0.349954,0.035491
...,...,...,...,...,...,...,...,...,...,...
1996,Airbnb CEO Brian Chesky predicts a wildly diff...,Airbnb CEO Brian Chesky. Mike Segar/Reuters Ai...,Airbnb CEO Brian Chesky. Mike Segar/Reuters\nA...,https://markets.businessinsider.com/news/stock...,Business Insider,1589083200000,Airbnb CEO Brian Chesky predicts a wildly diff...,0.303165,0.619250,0.077585
1997,Airbnb's cofounder makes a compelling argument...,People working in the lobby of the Airbnb head...,People working in the lobby of the Airbnb head...,https://markets.businessinsider.com/news/stock...,Business Insider,1581310800000,Airbnb's cofounder makes a compelling argument...,0.875880,0.062495,0.061625
1998,Airbnb banning 'party houses' after Orinda Hal...,"In response to the shooting in Orinda, Airbnb ...",In response to the shooting at a Halloween par...,https://www.mercurynews.com/2019/11/02/airbnb-...,The Mercury News,1572725309000,Airbnb banning 'party houses' after Orinda Hal...,0.644503,0.301102,0.054395
1999,Read the application form that got the 'Spotif...,Yunha Kim dropped out of Stanford Graduate Sch...,Yunha Kim dropped out of Stanford Graduate Sch...,https://markets.businessinsider.com/news/stock...,Business Insider,1557460800000,Read the application form that got the 'Spotif...,0.912416,0.022099,0.065484


In [10]:
df['dt'] = pd.to_datetime(df['dt'], unit='ms')
df['dt'] = df['dt'].dt.date

